## CZĘŚĆ 3: MODELOWANIE

1. Wczytanie danych, obróbka danych zgodnie z feature engineering

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import sklearn 

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.tree import DecisionTreeClassifier

In [45]:
# Wczytanie danych - dane do trenowania modelu
development_unit_df = pd.read_csv(".\\dane\\training_sample_team4.csv")

## Ramka danych walidacyjna dla zespołu budującego
test = pd.read_csv(".\\dane\\validation_sample_team4.csv")

In [46]:
scaler = MinMaxScaler()
standard = StandardScaler()

zgodnie z wskazówkami, outliery zostały zastąpione wartościami granicznymi

In [47]:
def handle_outliers(df):
    columns_to_be_checked = ['r', 'u', 'i', 'z', 'g']
    
    df = df[df['u'] > 0]

    for i in range(len(columns_to_be_checked)):
        column = df[columns_to_be_checked[i]]
        Q1 = column.quantile(0.25)
        Q3 = column.quantile(0.75)

        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = (column < lower_bound) | (column > upper_bound)

        outliers_count = outliers.sum()

        outliers_percentage = outliers.mean() * 100
        
        # Zastąpienie outlierów wartościami równymi Q1 - 1.5 * IQR lub Q3 + 1.5 * IQR
        df.loc[outliers, columns_to_be_checked[i]] = np.where(column[outliers] < lower_bound, lower_bound, upper_bound)

        ##print("Ilość outlierów dla kolumny ", columns_to_be_checked[i], ": ", outliers_count)
        ##print("Procentowy udział outlierów dla kolumny ", columns_to_be_checked[i], ": ", outliers_percentage)

    return df

In [48]:
df = handle_outliers(development_unit_df)
test = handle_outliers(test)

dla przypomnienia:
###### usunięto kolumny silnie skorelowane
###### filtry w kolorze czerwieni zostały zsumowane i zawarte w jednej, nowej kolumnie (r + i + z)
###### końcowo, do tworzenia modeli użyto kolumn związanych z filtrami oraz kolumnę 'redshift' - uznano, że wszystkie zmienne związane z id lub z położeniem obiektów (kąty alpha i delta) nie wpływają na efektywność modelu
###### kolumny zostały zestandaryzowane i/lub znormalizowane


In [49]:
def modify_data(df):
    df = df.loc[:, df.columns != 'rerun_ID']
    df = df.loc[:, df.columns != 'obj_ID']
    df = df.loc[:, df.columns != 'plate']
    
    df['r+i+z'] = df[['r', 'i', 'z']].sum(axis=1)

    # Normalizacja
    columns_to_normalize = ['run_ID', 'cam_col', 'spec_obj_ID', 'MJD', 'alpha', 'fiber_ID']
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

    # Standaryzacja
    columns_to_standardize = ['delta', 'u', 'g', 'r', 'i', 'z', 'redshift', 'field_ID', 'r+i+z']
    df[columns_to_standardize] = standard.fit_transform(df[columns_to_standardize])

    # Usuwanie kolumn
    df = df.drop(['MJD'], axis = 1)
    df = df.drop(['r', 'i', 'z'], axis = 1)

    # Odrzucenie cech na podstawie interpretacji ich znaczenia
    df = df.drop(['spec_obj_ID'], axis = 1) # numer obiektu
    df = df.drop(['alpha'], axis = 1) # współrzędne astronomiczne
    df = df.drop(['delta'], axis = 1) # współrzędne astronomiczne
    df = df.drop(['field_ID'], axis = 1) # numer pola ze zdjęcia (fragment zdjęcia)
    df = df.drop(['cam_col'], axis = 1) # numer kolumny ze zdjęcia (fragment zdjęcia)
    df = df.drop(['run_ID'], axis = 1) # numer zdjęcia
    df = df.drop(['fiber_ID'], axis = 1) # numer włókna, które skierowało światło na płaszczyznę ogniskową
    
    return df

In [50]:
df = modify_data(development_unit_df)

In [51]:
df.head()

,u,g,class,redshift,r+i+z
0,0.678738,0.564729,QSO,2.408604,1.427677
1,-1.548889,-1.533804,GALAXY,-0.730359,-1.239177
2,-1.744904,-2.029105,STAR,-0.790164,-2.116429
3,-1.006758,-1.066027,GALAXY,-0.677478,-0.936643
4,-1.228689,-1.217197,GALAXY,-0.705030,-0.941500


In [52]:
test = modify_data(test)

In [53]:
test.head()

,u,g,class,redshift,r+i+z
0,0.695839,0.356947,QSO,2.206901,1.031172
1,0.181778,0.253523,GALAXY,-0.706093,0.985689
2,-0.366130,0.220645,QSO,2.013251,0.860169
3,0.756569,0.622013,STAR,-0.786839,-0.020183
4,0.764057,0.794126,GALAXY,-0.010139,0.314667


In [54]:
train = df.copy()
t = test.copy()

### 2. Modelowanie bez strojenia hiperparametrów: 
#### Użyto 7 modeli klasyfikacyjnych, następnie na podstawie ich predykcyjności wybrano modele do dalszej obróbki

Model 1 - KNeighboursClassifier

In [55]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def evaluate_knn_model(train_data, train_target, test_data, test_target):
    # Inicjalizacja modelu KNN z domyślnymi parametrami
    knn_model = KNeighborsClassifier()
    
    # Trenowanie modelu na danych treningowych
    knn_model.fit(train_data, train_target)
    
    # Predykcja klas dla danych testowych
    test_predictions = knn_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = knn_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
##train_target_encoded = encoder.fit_transform(train[['class']])
#test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_knn_model(train_data, train_target, test_data, test_target)


Dokładność całkowita (Accuracy): 0.9615238095238096

Raport klasyfikacji:
              precision    recall  f1-score   support

      GALAXY       0.97      0.96      0.97     12538
         QSO       0.96      0.93      0.94      3973
        STAR       0.93      0.99      0.96      4489

    accuracy                           0.96     21000
   macro avg       0.95      0.96      0.96     21000
weighted avg       0.96      0.96      0.96     21000


Confusion matrix:
[[12059   166   313]
 [  281  3687     5]
 [   40     3  4446]]
AUC dla klasy GALAXY: 0.9797728165653182
Gini dla klasy GALAXY: 0.9595456331306365
 
AUC dla klasy QSO: 0.978214195600062
Gini dla klasy QSO: 0.9564283912001239
 
AUC dla klasy STAR: 0.9939763724215583
Gini dla klasy STAR: 0.9879527448431167
 
Średni AUC: 0.9839877948623128
Gini coefficient: 0.9679755897246256


Model 2 - RandomForestClassifier

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_forest_model(train_data, train_target, test_data, test_target):
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Trenowanie modelu
    rf_model.fit(train_data, train_target)

    # Predykcja klas dla danych testowych
    test_predictions = rf_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = rf_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
#train_target_encoded = encoder.fit_transform(train[['class']])
#test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_forest_model(train_data, train_target, test_data, test_target)

Dokładność całkowita (Accuracy): 0.9635714285714285

Raport klasyfikacji:
              precision    recall  f1-score   support

      GALAXY       0.96      0.98      0.97     12538
         QSO       0.95      0.93      0.94      3973
        STAR       0.98      0.96      0.97      4489

    accuracy                           0.96     21000
   macro avg       0.96      0.95      0.96     21000
weighted avg       0.96      0.96      0.96     21000


Confusion matrix:
[[12255   174   109]
 [  283  3689     1]
 [  198     0  4291]]
AUC dla klasy GALAXY: 0.9892064121289668
Gini dla klasy GALAXY: 0.9784128242579335
 
AUC dla klasy QSO: 0.9883353338032839
Gini dla klasy QSO: 0.9766706676065677
 
AUC dla klasy STAR: 0.9975750655250132
Gini dla klasy STAR: 0.9951501310500264
 
Średni AUC: 0.991705603819088
Gini coefficient: 0.983411207638176


Model 3

xboost

In [57]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_xboost_model(train_data, train_target, test_data, test_target):

    xgb_model = XGBClassifier(n_estimators=100, random_state=42)

    # Trenowanie modelu XGBoost
    xgb_model.fit(train_data, train_target)

    # Predykcja klas dla danych testowych
    test_predictions = xgb_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    #print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = xgb_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
encoder = OneHotEncoder(sparse=False)
train_target_encoded = encoder.fit_transform(train[['class']])
test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_xboost_model(train_data, train_target_encoded, test_data, test_target_encoded)

Dokładność całkowita (Accuracy): 0.9452857142857143

Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     12538
           1       0.95      0.93      0.94      3973
           2       0.97      0.92      0.95      4489

   micro avg       0.95      0.96      0.95     21000
   macro avg       0.96      0.94      0.95     21000
weighted avg       0.95      0.96      0.95     21000
 samples avg       0.95      0.96      0.95     21000


Confusion matrix:
AUC dla klasy 0: 0.9880172783365371
Gini dla klasy 0: 0.9760345566730741
 
AUC dla klasy 1: 0.9918030499256958
Gini dla klasy 1: 0.9836060998513916
 
AUC dla klasy 2: 0.9965855809230592
Gini dla klasy 2: 0.9931711618461183
 
Średni AUC: 0.992135303061764
Gini coefficient: 0.984270606123528


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model 4

Gradient Boosting

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
def evaluate_gradient_model(train_data, train_target, test_data, test_target):

    gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

    # Trenowanie modelu Gradient Boosting
    gb_model.fit(train_data, train_target)


    # Predykcja klas dla danych testowych
    test_predictions = gb_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = gb_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
#train_target_encoded = encoder.fit_transform(train[['class']])
#test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_gradient_model(train_data, train_target, test_data, test_target)

Dokładność całkowita (Accuracy): 0.9655714285714285

Raport klasyfikacji:
              precision    recall  f1-score   support

      GALAXY       0.96      0.98      0.97     12538
         QSO       0.96      0.91      0.94      3973
        STAR       0.98      0.97      0.97      4489

    accuracy                           0.97     21000
   macro avg       0.97      0.96      0.96     21000
weighted avg       0.97      0.97      0.97     21000


Confusion matrix:
[[12275   154   109]
 [  341  3631     1]
 [  118     0  4371]]
AUC dla klasy GALAXY: 0.9888525363631973
Gini dla klasy GALAXY: 0.9777050727263945
 
AUC dla klasy QSO: 0.9912534571652245
Gini dla klasy QSO: 0.982506914330449
 
AUC dla klasy STAR: 0.9963079488553632
Gini dla klasy STAR: 0.9926158977107264
 
Średni AUC: 0.992137980794595
Gini coefficient: 0.9842759615891901


Model 5: Regresja logistyczna

In [59]:
def evaluate_regression_model(train_data, train_target, test_data, test_target):

    log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
    log_reg_model.fit(train_data, train_target)


    # Predykcja klas dla danych testowych
    test_predictions = log_reg_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = log_reg_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
#train_target_encoded = encoder.fit_transform(train[['class']])
#test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_regression_model(train_data, train_target, test_data, test_target)

Dokładność całkowita (Accuracy): 0.9533333333333334

Raport klasyfikacji:
              precision    recall  f1-score   support

      GALAXY       0.96      0.96      0.96     12538
         QSO       0.94      0.88      0.91      3973
        STAR       0.94      1.00      0.97      4489

    accuracy                           0.95     21000
   macro avg       0.95      0.95      0.95     21000
weighted avg       0.95      0.95      0.95     21000


Confusion matrix:
[[12053   205   280]
 [  490  3480     3]
 [    0     2  4487]]
AUC dla klasy GALAXY: 0.9830697143458644
Gini dla klasy GALAXY: 0.9661394286917289
 
AUC dla klasy QSO: 0.9850534243513777
Gini dla klasy QSO: 0.9701068487027553
 
AUC dla klasy STAR: 0.9956065121615257
Gini dla klasy STAR: 0.9912130243230515
 
Średni AUC: 0.9879098836195892
Gini coefficient: 0.9758197672391784


Model 6: Model Markova

In [60]:
def evaluate_markov_model(train_data, train_target, test_data, test_target):
    
    markov_model = ClassifierChain(base_estimator=DecisionTreeClassifier(), order='random', random_state=42)
    markov_model.fit(train_data, train_target)


    # Predykcja klas dla danych testowych
    test_predictions = markov_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    #print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = markov_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
encoder = OneHotEncoder(sparse=False)
train_target_encoded = encoder.fit_transform(train[['class']])
test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_markov_model(train_data, train_target_encoded, test_data, test_target_encoded)

Dokładność całkowita (Accuracy): 0.897952380952381

Raport klasyfikacji:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94     12538
           1       0.79      0.92      0.85      3973
           2       0.97      0.71      0.82      4489

   micro avg       0.90      0.90      0.90     21000
   macro avg       0.89      0.86      0.87     21000
weighted avg       0.91      0.90      0.90     21000
 samples avg       0.90      0.90      0.90     21000


Confusion matrix:
AUC dla klasy 0: 0.9175784179083061
Gini dla klasy 0: 0.8351568358166122
 
AUC dla klasy 1: 0.9293184891599078
Gini dla klasy 1: 0.8586369783198156
 
AUC dla klasy 2: 0.8522944511134757
Gini dla klasy 2: 0.7045889022269514
 
Średni AUC: 0.8997304527272298
Gini coefficient: 0.7994609054544597


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model 7

In [ ]:
Adaptacyjne drzewa decyzyjne

In [61]:
def evaluate_decisiontree_model(train_data, train_target, test_data, test_target):

    
    adtree_model = DecisionTreeClassifier()
    adtree_model.fit(train_data, train_target)


    # Predykcja klas dla danych testowych
    test_predictions = adtree_model.predict(test_data)
    
    # Dokładność całkowita
    accuracy = accuracy_score(test_target, test_predictions)
    print("Dokładność całkowita (Accuracy):", accuracy)
    
    # Raport klasyfikacji
    print("\nRaport klasyfikacji:")
    print(classification_report(test_target, test_predictions))
    
    # Macierz pomyłek
    print("\nConfusion matrix:")
    print(confusion_matrix(test_target, test_predictions))
    
    proba_predictions = adtree_model.predict_proba(test_data)
    
    # Konwersja etykiet klas na formę binarną dla każdej klasy
    lb = LabelBinarizer()
    lb.fit(train_target)
    test_target_bin = lb.transform(test_target)
    
    # Obliczenie AUC dla każdej klasy
    auc_scores = []
    for i in range(len(lb.classes_)):
        auc_score = roc_auc_score(test_target_bin[:, i], proba_predictions[:, i])
        auc_scores.append(auc_score)
        print(f"AUC dla klasy {lb.classes_[i]}: {auc_score}")
        print(f"Gini dla klasy {lb.classes_[i]}: {2 * auc_score - 1}")
        print(" ")
    
    # Średni AUC
    mean_auc = np.mean(auc_scores)
    print("Średni AUC:", mean_auc)
    
    gini_coefficient = 2 * mean_auc - 1
    print("Gini coefficient:", gini_coefficient)

# Przykładowe dane
train_data = train.drop('class', axis=1)
train_target = train['class']
test_data = t.drop('class', axis=1)
test_target = t['class']

# Zakodowanie klas za pomocą OneHotEncoder
#encoder = OneHotEncoder(sparse=False)
#train_target_encoded = encoder.fit_transform(train[['class']])
#test_target_encoded = encoder.transform(t[['class']])

# Ocena modelu
evaluate_decisiontree_model(train_data, train_target, test_data, test_target)

Dokładność całkowita (Accuracy): 0.9015238095238095

Raport klasyfikacji:
              precision    recall  f1-score   support

      GALAXY       0.93      0.96      0.95     12538
         QSO       0.77      0.92      0.84      3973
        STAR       0.97      0.72      0.83      4489

    accuracy                           0.90     21000
   macro avg       0.89      0.87      0.87     21000
weighted avg       0.91      0.90      0.90     21000


Confusion matrix:
[[12045   397    96]
 [  325  3647     1]
 [  563   686  3240]]
AUC dla klasy GALAXY: 0.9278699018279161
Gini dla klasy GALAXY: 0.8557398036558321
 
AUC dla klasy QSO: 0.9271706367779894
Gini dla klasy QSO: 0.8543412735559788
 
AUC dla klasy STAR: 0.8579447207333064
Gini dla klasy STAR: 0.7158894414666128
 
Średni AUC: 0.9043284197797373
Gini coefficient: 0.8086568395594746


#### wnioski:

1) najwyższa dokładność ogólna: knn, randomForest, gradient boosting ~ 0.96


2) najwyższa dokładność:


        dla Star: randomForest, gradient boosting ~ 0.98
        
        dla Galaxy: knn ~ 0.97
        
        dla Quasar: knn, gradient boosting ~ 0.96
        
        
3) najwyższy średni auc: gradient, boosting, xboost, randomForest  ~ 0.99


4) najwyższy auc:

        dla Star: wszystkie oprócz Markowa i DecisionTreeClassifier ~ 0.99
        
        dla Galaxy: wszystkie oprócz Markowa i DecisionTreeClassifier ~ 0.99
        
        dla Quasar: xboost, gradient boosting ~ 0.99


najsłabiej sobie radzi Model Markova

Decision Tree Classifier bardzo słabo przewiduję klasę STAR, dla każdego innego modelu predykcyjność STAR jest najwyższa, często bliska 100 %

### 3) Strojenie hiperparametrów dla wybranych modeli, wybór najlepszego

#### Do kolejnej części wybrano te modele, które osiągnęły najlepsze wyniki z standardowymi, niezmienionymi parametrami. Są to:

1) Gradient Boosting

2) Random Forest Classifier

3) KNeighbours Classifier